# Placing Drums in Video Feed

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import imutils
import pygame
import pymunk

pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
floor_tom = cv2.imread("tom-tom.jpg")
floor_tom = imutils.resize(floor_tom, height=220)
    
medium_tom = cv2.imread("bass.jpg")
medium_tom = imutils.resize(medium_tom, height = 220)
                        
snare = cv2.imread("snare.jpg")
snare = imutils.resize(snare, height = 220)
snare = imutils.rotate(snare, 45)

cymbal = cv2.imread("cymbal.jpg")
cymbal = cv2.resize(cymbal, (220,220))
cymbal = imutils.rotate(cymbal, -45)
   

In [3]:
def remove_bg(image):
    img = np.zeros(image.shape, dtype = np.uint8)
    img.fill(255)
    foreground = cv2.absdiff(image, img)
    foreground[foreground > 5] = image[foreground > 5]
    return foreground

In [4]:
space = pymunk.Space()

# Defined 4 static segments to be represented by 4 drum pieces giving it collision_type 1

cym_body = pymunk.Body(body_type=pymunk.Body.STATIC)
cym_line = pymunk.Segment(cym_body, (85, 335), (225, 475), 5)
cym_line.collision_type = 1
tomtom_body = pymunk.Body(body_type=pymunk.Body.STATIC)
tomtom_line = pymunk.Segment(tomtom_body, (280, 510), (440, 510), 5)
tomtom_line.collision_type = 1
bass_body = pymunk.Body(body_type=pymunk.Body.STATIC)
bass_line = pymunk.Segment(bass_body, (520, 510), (680, 510), 5)
bass_line.collision_type = 1
snare_body = pymunk.Body(body_type=pymunk.Body.STATIC)
snare_line = pymunk.Segment(snare_body, (735, 475), (875, 335), 5)
snare_line.collision_type = 1

space.add(cym_body, cym_line)
space.add(tomtom_body, tomtom_line)
space.add(bass_body, bass_line)
space.add(snare_body, snare_line)

In [5]:
# Defined two circular pymunk bodies to be represented by fingers giving it collision_type 2

radius = 20

left_index = pymunk.Body(10, 1666, body_type=pymunk.Body.KINEMATIC)
left_shape = pymunk.Circle(left_index ,radius)
left_shape.collision_type = 2
space.add(left_index, left_shape)

right_index = pymunk.Body(10, 1666, body_type=pymunk.Body.KINEMATIC)
right_shape = pymunk.Circle(right_index, radius)
right_shape.collision_type = 2
space.add(right_index, right_shape)

In [6]:
# Defined method for begin attribute of pymunk collision handler

def collision(arbiter, space, data):
    if (arbiter.shapes[0].a == cym_line.a):
        if (arbiter.shapes[1].body.velocity[1] > 0):
            sound = pygame.mixer.Sound(sounds[0])
            sound.set_volume(arbiter.total_ke)
            sound.play()
    elif (arbiter.shapes[0].a == tomtom_line.a):
        if (arbiter.shapes[1].body.velocity[1] > 0):
            sound = pygame.mixer.Sound(sounds[1])
            sound.set_volume(arbiter.total_ke)
            sound.play()
    elif (arbiter.shapes[0].a == bass_line.a):
        if (arbiter.shapes[1].body.velocity[1] > 0):
            sound = pygame.mixer.Sound(sounds[2])
            sound.set_volume(arbiter.total_ke)
            sound.play()
    elif (arbiter.shapes[0].a == snare_line.a):
        if (arbiter.shapes[1].body.velocity[1] > 0):
            sound = pygame.mixer.Sound(sounds[3])
            sound.set_volume(arbiter.total_ke)
            sound.play()
    return True

In [7]:
pygame.mixer.pre_init(22050, -16, 2, 512)
pygame.init()
pygame.mixer.quit()
pygame.mixer.init(22050, -16, 2, 1024)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

cx1 , cy1, cx2, cy2 = 0, 0, 0, 0

sounds = ['Crash-Cymbal-Hit.mp3','Floor-Tom-Drum.mp3', 'Medium-Tom-Drum.mp3', 'Snare-Drum.mp3']

feed = cv2.VideoCapture(0)

while(True):
    
    # Capturing Frames
    ret, frame = feed.read()
    frame = imutils.resize(frame, width=960)
    frame = cv2.flip(frame,1)
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
    
    # Placing Drum inside the frame
    
    floor_tom = remove_bg(floor_tom)
    medium_tom = remove_bg(medium_tom)
    snare = remove_bg(snare)
    cymbal = remove_bg(cymbal)
    
    height, width, _ = frame.shape
    frame[450:670, 250:470][floor_tom > 5] = floor_tom[floor_tom > 5]
    frame[450:670, 490:710][medium_tom > 5] = medium_tom[medium_tom > 5]
    frame[320:540, 720:940][snare > 5] = snare[snare > 5]
    frame[320:540, 20:240][cymbal > 5] = cymbal[cymbal > 5]
    
    handler = space.add_collision_handler(1,2)
    
    # Hand Tracking 
    results = hands.process(frameRGB)

    if results.multi_hand_landmarks:
        for index, handLms in enumerate(results.multi_hand_landmarks):
            
            # Left Right detection using mediapipe multi handedness
            if results.multi_handedness[index]:
                
                if results.multi_handedness[index].classification[0].label == 'Left':
                    for i, lm in enumerate(handLms.landmark):
                        if i == 8:
                            cx1, cy1 = int(lm.x*width), int(lm.y*height)
                            left_vel_x = cx1 - left_index.position[0]
                            left_vel_y = cy1 - left_index.position[1]
                            left_index.velocity = 20.0*left_vel_x, 20.0*left_vel_y
                            
                if results.multi_handedness[index].classification[0].label == 'Right':
                    for i, lm in enumerate(handLms.landmark):
                        if i == 8:
                            cx2, cy2 = int(lm.x*width), int(lm.y*height)
                            right_vel_x = cx2 - right_index.position[0]
                            right_vel_y = cy2 - right_index.position[1]
                            right_index.velocity = 20.0*right_vel_x, 20.0*right_vel_y
            
            
            # Added pymunk collision _handler to the space for collision type 1 (segment) and 2 (fingers)
            
            handler.begin = collision
            space.step(0.02)
    
    cv2.imshow('frame', frame)
    
    # Close if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
feed.release()
cv2.destroyAllWindows()
